In [ ]:
from PIL import Image
import numpy as np

In [ ]:
'''
So basically,
A = 8 i.e. 8 diff views in 8 rows
n = 10 i.e. 10 different objects corresponding to a view
'''
X = np.zeros((11,32,32))
for view in range(8):
  #initialize current view with a 32x32 matrix
  curr_view = np.zeros((32,32))
  
  for object in range(1,11):
    
    img = np.array(Image.open('obj'+str(object)+'__'+str(view)+'.png'))
    # print('image', img.shape)
    curr_view = np.append(curr_view, img, axis = 0)

  # print(curr_view.shape)
  # now curr_view is in 11 128x128 matrix form
  curr_view = np.reshape(curr_view, (11,32,32))
  curr_view = curr_view[1:]
  # print(curr_view.shape)

  X = np.append(X, curr_view, axis = 0)
print(X.shape)
X = X[11:]
X = np.reshape(X, (8,10,32,32))

(91, 32, 32)


In [ ]:
X.shape

(8, 10, 32, 32)

Compute O:

In [ ]:
# first calculate L
# print(X[0].shape) # -> (10, 32, 32)
# first convert to 2D matrix for matrix multiplication in L

#32x32 = 10245
X_2d =  np.zeros((10,1024))
for view in range(8):
  temp = np.reshape(X[view], (10,1024))
  X_2d = np.append(X_2d, temp, axis = 0)


# removing the first zero matrix
X_2d = X_2d[10:]

X_2d = np.reshape(X_2d, (8,10,1024))
print(X_2d.shape)
print(X_2d[0].shape) 

(8, 10, 1024)
(10, 1024)


In [ ]:
# now rows of X_2d will be used as 2d matrix for matrix multiplication in 
# caculating L

# first initialize L as follows:
L = np.zeros((8, 1024,1024))
for view in range(8):
  L = np.append(L, np.zeros((8, 1024,1024)), axis = 0)
L = L[8:]
L = np.reshape(L, (8,8,1024,1024))
print(L.shape)

#sparse matrix api

(8, 8, 1024, 1024)


In [ ]:
'''
Now calculate X1.X1' ..... XA.XA' and put it in L
shape of L = (8, 8, 1024, 1024)

first view position = L[0][0]
second view position = L[1][1]
... upto A

So substitute a 1024x1024 matrix
'''
for view in range(8):

  # dimension of Xa -> (10, 1024)
  Xa = X_2d[view]
  # dimension of Xa' -> (1024, 10)
  Xa_t = np.transpose(Xa)

  # d_element -> multiply Xa and Xa'
  d_element = np.matmul(Xa_t, Xa)
  L[view][view] = d_element


L = np.reshape(L,(1024*8, 1024*8))
print(L.shape)

(8192, 8192)


In [ ]:
'''
Let gamma be 0.9

Now calculate the term 1 in the multiplication
'''
gamma = 0.9
I = np.identity(8192)

term1 = np.add(L, gamma*I)

#finally take inverse
term1 = np.linalg.inv(term1)

In [ ]:
term1.shape

(8192, 8192)

In [ ]:
'''
Calculate term 2 of O

Dimensions would be as follows:

Xa = 10 x 1024
Xa' = 1024 x 10
X = (8,10,1024) -> (10,1024*8) -> (10, 8192)

Multiplication of above three terms = 10 x 1024

On multiplication with outer X', finally we get 1024 x 1024 matrix
'''
sum = np.zeros((10,8192))

#calculating the summation
for view in range(8):
  
  # dimension of Xa -> (10, 1024)
  Xa = X_2d[view]

  # dimension of Xa' -> (1024, 10)
  Xa_t = np.transpose(Xa)
  
  X_sum = np.reshape(X_2d, (10,8192))
  
  sum = np.add(sum,np.matmul(np.matmul(Xa,Xa_t),X_sum))

temp_X2d = np.reshape(X_2d, (10,8192))

#calculating the term 2
term2 = np.matmul(np.transpose(temp_X2d), sum)
print(term2.shape)

(8192, 8192)


In [ ]:
O = np.matmul(term1, term2)

In [ ]:
O.shape

(8192, 8192)

In [ ]:
O

array([[ 1.66745020e+23,  1.66745020e+23,  1.66745020e+23, ...,
         3.67001059e+23,  3.67001059e+23,  3.67001059e+23],
       [-6.17270414e+22, -6.17270414e+22, -6.17270414e+22, ...,
        -1.34483520e+23, -1.34483520e+23, -1.34483520e+23],
       [ 5.48377964e+23,  5.48377964e+23,  5.48377964e+23, ...,
         1.20433882e+24,  1.20433882e+24,  1.20433882e+24],
       ...,
       [-7.18348498e+23, -7.18348498e+23, -7.18348498e+23, ...,
        -1.56901194e+24, -1.56901194e+24, -1.56901194e+24],
       [ 6.93932227e+23,  6.93932227e+23,  6.93932227e+23, ...,
         1.51895220e+24,  1.51895220e+24,  1.51895220e+24],
       [ 1.78150285e+23,  1.78150285e+23,  1.78150285e+23, ...,
         3.88444236e+23,  3.88444236e+23,  3.88444236e+23]])

In [ ]:
eig_value, eig_vectors = np.linalg.eig(O)

In [ ]:
eig_value.shape

(8192,)

In [ ]:
eig_vectors.shape

(8192, 8192)

In [ ]:
#choose first 8 eigen vectors for V = 8
P = eig_vectors[:8,:40]

# Now multiply
print(P[0,:].shape)


(40,)


In [ ]:
Z = np.zeros((8,256))
for view in range(8):
  Xa = X_2d[view]
  Xa = np.reshape(Xa, (256,40))
  Pa = P[view,:]
  temp = np.matmul(Xa,Pa)

  Z = np.add(Z,temp)
print(Z)


[[ 11.6770626  -2.01302228j  -0.35263939 +0.10846071j
  109.3426459 -15.45326445j ...  -7.80441072 +3.54567389j
   25.88616234 -1.66109872j  51.88017576 -1.02359664j]
 [ 11.6770626  -2.01302228j  -0.35263939 +0.10846071j
  109.3426459 -15.45326445j ...  -7.80441072 +3.54567389j
   25.88616234 -1.66109872j  51.88017576 -1.02359664j]
 [ 11.6770626  -2.01302228j  -0.35263939 +0.10846071j
  109.3426459 -15.45326445j ...  -7.80441072 +3.54567389j
   25.88616234 -1.66109872j  51.88017576 -1.02359664j]
 ...
 [ 11.6770626  -2.01302228j  -0.35263939 +0.10846071j
  109.3426459 -15.45326445j ...  -7.80441072 +3.54567389j
   25.88616234 -1.66109872j  51.88017576 -1.02359664j]
 [ 11.6770626  -2.01302228j  -0.35263939 +0.10846071j
  109.3426459 -15.45326445j ...  -7.80441072 +3.54567389j
   25.88616234 -1.66109872j  51.88017576 -1.02359664j]
 [ 11.6770626  -2.01302228j  -0.35263939 +0.10846071j
  109.3426459 -15.45326445j ...  -7.80441072 +3.54567389j
   25.88616234 -1.66109872j  51.88017576 -1.0235